In [1]:
import pandas as pd 
import torch 
import numpy as np 
import torch.optim as optim
from torch.utils.data import Dataset , DataLoader , random_split
import torchvision.transforms as transforms
from PIL import Image
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import torch.nn as nn

In [2]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
print(device)

cuda


In [3]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [4]:
full_dataset = ImageFolder(root='Pet_Images' , transform=train_transform)

In [5]:
train_size =int( 0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

In [6]:
train_dataset , test_dataset = random_split(full_dataset , [train_size , test_size])

In [7]:
len(train_dataset)

19967

In [8]:
len(test_dataset)

4992

In [9]:
train_loader = DataLoader(train_dataset , batch_size=32 , shuffle=True , pin_memory=True)
test_loader  = DataLoader(test_dataset , batch_size=32 , shuffle=True , pin_memory=True)

In [39]:
class mynn(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3 , 32 , kernel_size=(3,3) , padding='same'),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2) , stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)            
        )
        self.fc_layers = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 14 * 14, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 2)  
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1) 
        x = self.fc_layers(x)
        return x


In [41]:
model = mynn().to(device)

In [43]:
print(model)

mynn(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

In [45]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters() , lr=0.01 , weight_decay=1e-4)

In [47]:
def train(model, dataloader, criterion, optimizer, device):
    # Set the model to training mode
    model.train()
    
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    # Loop through all batches of data
    for inputs, labels in dataloader:
        # Move the data to the selected device (CPU or GPU)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Clear the previous gradients
        optimizer.zero_grad()

        # Forward pass: make predictions
        outputs = model(inputs)

        # Calculate the loss
        loss = criterion(outputs, labels)

        # Backward pass: compute gradients
        loss.backward()

        # Update the model's weights
        optimizer.step()

        # Add this batch's loss to the total loss
        total_loss += loss.item() * inputs.size(0)

        # Get the predicted class labels
        _, predicted = torch.max(outputs, 1)

        # Update total number of samples and correct predictions
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

    # Calculate average loss and accuracy for the epoch
    average_loss = total_loss / len(dataloader.dataset)
    accuracy = total_correct / total_samples

    return average_loss, accuracy


In [49]:
num_epochs = 10
train_losses = []
train_accs = []
test_losses = []
test_accs = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    # Train
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # Evaluate
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    test_losses.append(test_loss)
    test_accs.append(test_acc)
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
    print("-" * 50)

Epoch 1/10


c:\Users\Zeshan\anaconda3\Lib\site-packages\PIL\TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


NameError: name 'evaluate' is not defined